## Joining and Appending DataFrames

### Import Pyspark and create SparkSession.

This is the first thing to do when working with pyspark. The spark variable will also provide access to a UI to monitor jobs.

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Joining").getOrCreate()

#### Creating sample dataframes

In [2]:
valuesP = [('koala',1,'yes'),('caterpillar',2,'yes'),('deer',3,'yes'),('human',4,'yes')]
eats_plants = spark.createDataFrame(valuesP,['name','id','eats_plants'])

valuesM = [('shark',5,'yes'),('lion',6,'yes'),('tiger',7,'yes'),('human',4,'yes')]
eats_meat = spark.createDataFrame(valuesM,['name','id','eats_meat'])

print("Plant eaters (herbivores)")
print(eats_plants.show())
print("Meat eaters (carnivores)")
print(eats_meat.show())

Plant eaters (herbivores)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
Meat eaters (carnivores)
+-----+---+---------+
| name| id|eats_meat|
+-----+---+---------+
|shark|  5|      yes|
| lion|  6|      yes|
|tiger|  7|      yes|
|human|  4|      yes|
+-----+---+---------+

None


In [3]:
new_df = eats_plants

#### Union of two dataframes (columns should be same with same datatype, stacking them over rows

In [6]:
df_append = eats_plants.union(new_df)

In [7]:
df_append.count(),eats_plants.count()

(8, 4)

### Inner Join (Keep rows with common values of key in both dataframes)

In [8]:
inner_join = eats_plants.join(eats_meat,["name","id"],"inner")
inner_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



In [9]:
inner_join = eats_plants.join(eats_meat,["name",],"inner")
inner_join.show()

+-----+---+-----------+---+---------+
| name| id|eats_plants| id|eats_meat|
+-----+---+-----------+---+---------+
|human|  4|        yes|  4|      yes|
+-----+---+-----------+---+---------+



#### Left Join (join dataframes with common values of key in both and all the values of key in left dataframe)

In [10]:
left_join = eats_plants.join(eats_meat,["name","id"],"left")
left_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



#### Right Join (join dataframes with common values of key in both and all the values of key in right dataframe)

In [11]:
right_join = eats_plants.join(eats_meat,["name","id"],"right")
right_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|shark|  5|       null|      yes|
|human|  4|        yes|      yes|
|tiger|  7|       null|      yes|
| lion|  6|       null|      yes|
+-----+---+-----------+---------+



#### Left Join dataframes and filter it based on Null values of column

In [13]:
cond_join = eats_plants.join(eats_meat,["name","id"],"left").filter(eats_meat.name.isNull())
cond_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



#### Full outer join

In [14]:
full_join = eats_plants.join(eats_meat,["name","id"],"full")
full_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      shark|  5|       null|      yes|
|      human|  4|        yes|      yes|
|      tiger|  7|       null|      yes|
|       lion|  6|       null|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



In [17]:
import os
path = 'Datasets/uw-madison-courses/'


### Reading multiple CSVs in dataframe and assigning it a variable name

In [20]:
df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".")
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        

In [21]:
teachings

DataFrame[instructor_id: int, section_uuid: string]

In [22]:
df_list

['teachings',
 'course_offerings',
 'subject_memberships',
 'sections',
 'schedules',
 'rooms',
 'instructors',
 'grade_distributions',
 'subjects',
 'courses']

In [23]:
subjects.show()

+----+--------------------+------------+
|code|                name|abbreviation|
+----+--------------------+------------+
| 908|        Soil Science|    SOIL SCI|
| 350|             ENGLISH|     ENGLISH|
| 351|English as a Seco...|         ESL|
| 230|Human Development...|        HDFS|
| 352|             English|        ENGL|
| 231|    General Business|     GEN BUS|
| 232|Accounting and In...|    ACCT I S|
| 112|Biological System...|         BSE|
| 233|Finance, Investme...|     FINANCE|
| 476|        Horticulture|        HORT|
| 355|          Entomology|       ENTOM|
| 234| Information Systems|    INFO SYS|
| 235|International Bus...|    INTL BUS|
| 236|Management and Hu...|       M H R|
| 237|           Marketing|    MARKETNG|
| 359|      Design Studies|          DS|
| 238|Operations and Te...|         OTM|
| 239|Real Estate and U...|    REAL EST|
| 912|Spanish (Spanish ...|     SPANISH|
| 480|      Human Oncology|     H ONCOL|
+----+--------------------+------------+
only showing top

#### False stops show from truncating the colum  value

In [24]:
course_offerings.show(1,False)

+------------------------------------+------------------------------------+---------+--------------------------+
|uuid                                |course_uuid                         |term_code|name                      |
+------------------------------------+------------------------------------+---------+--------------------------+
|344b3ebe-da7e-314c-83ed-9425269695fd|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1092     |Cooperative Education Prog|
+------------------------------------+------------------------------------+---------+--------------------------+
only showing top 1 row



In [25]:
instructors.show(1,False)

+------+----------------+
|id    |name            |
+------+----------------+
|761703|JOHN ARCHAMBAULT|
+------+----------------+
only showing top 1 row



## Multi-step Joins: Here we are looking to get the instructor name and the courses they teach

Step1: get the instructor_id and name of the professor teaching a section by left joining teachings and instructors on instructor_id
Step2: get the course_offering_id and name of the professor by left joining step1 and sections on section_uuid
Step3: rename column name as instructor and inner joining step2 and course_offerings on course_offerin_uuid to get the name of instructor and the course they teach

In [26]:
step1 = teachings.join(instructors,teachings.instructor_id==instructors.id,"left").select("instructor_id","section_uuid","name")
step1.show()

+-------------+--------------------+----------------+
|instructor_id|        section_uuid|            name|
+-------------+--------------------+----------------+
|       761703|45adf63c-48c9-365...|JOHN ARCHAMBAULT|
|       761703|c6280e23-5e43-385...|JOHN ARCHAMBAULT|
|       761703|9395dc21-15d1-3fa...|JOHN ARCHAMBAULT|
|      3677061|b99e440b-39db-350...|  STEPHANIE KANN|
|       761703|ca1c841f-41d5-329...|JOHN ARCHAMBAULT|
|      3677061|da41b0aa-2b81-378...|  STEPHANIE KANN|
|       761703|51c4dc00-1fc7-3c7...|JOHN ARCHAMBAULT|
|       761703|53f95c0f-4ea9-374...|JOHN ARCHAMBAULT|
|       761703|574d9b35-9c76-338...|JOHN ARCHAMBAULT|
|       761703|8fc362a6-d94e-3ad...|JOHN ARCHAMBAULT|
|       761703|d50e7478-e12f-363...|JOHN ARCHAMBAULT|
|       761703|b21564d2-2bc4-3b8...|JOHN ARCHAMBAULT|
|       788586|8853d23e-64b1-3cf...|      KATHY PREM|
|       788586|94339199-eaf3-393...|      KATHY PREM|
|       761703|0d72958a-291b-33d...|JOHN ARCHAMBAULT|
|       761703|dfb55b25-ac7a

In [27]:
step2 = step1.join(sections,step1.section_uuid==sections.uuid,'left').select("name","course_offering_uuid")
step2.limit(4).toPandas()

,name,course_offering_uuid
0,THOMAS JAHNS,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,db253216-2e66-3267-86b2-7b9f5fe07223


In [28]:
step3 = step2.withColumnRenamed("name","instructor").join(course_offerings,step2.course_offering_uuid==course_offerings.uuid,"inner")
step3.limit(4).toPandas()

,instructor,course_offering_uuid,uuid,course_uuid,term_code,name
0,THOMAS JAHNS,f850ab24-740c-311a-a669-804a3fea7b0b,f850ab24-740c-311a-a669-804a3fea7b0b,2c3a2a38-9f53-3cfb-8f37-cde4cb3d4b4b,1074,Master's Research or Thesis
1,JEAN-FRANCOIS HOUDE,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef,622a73dc-c070-38e4-8ba8-2b2c3f7a1056,1082,Wrkshp-Industrl Organizatn
2,CHRISTOPHER R TABER,3beb7bd7-4877-3c63-8afc-62f8b74e72fc,3beb7bd7-4877-3c63-8afc-62f8b74e72fc,685a727c-fdc0-3ba6-b521-1f236f69b5fa,1164,Workshop - Public Economics
3,MARISA S OTEGUI,db253216-2e66-3267-86b2-7b9f5fe07223,db253216-2e66-3267-86b2-7b9f5fe07223,abc31e90-8a8f-37bf-9fe0-bacd53aa2a1f,1084,Plant Cell Biology


In [29]:
from pyspark.sql.functions import levenshtein

#### Levenshtein distance is the distance between two strings such that changing string1 with number of distance to get string2

In [30]:
df0 = spark.createDataFrame([('Aple', 'Apple','Microsoft','IBM')], ['Input', 'Option1','Option2','Option3'])
print("Correct this company name: Aple")
df0.select(levenshtein('Input', 'Option1').alias('Apple')).show()
df0.select(levenshtein('Input', 'Option2').alias('Microsoft')).show()
df0.select(levenshtein('Input', 'Option3').alias('IBM')).show()

Correct this company name: Aple
+-----+
|Apple|
+-----+
|    1|
+-----+

+---------+
|Microsoft|
+---------+
|        9|
+---------+

+---+
|IBM|
+---+
|  4|
+---+

